In [80]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['kdd-cup-10percent', 'kdd-labels']


In [81]:
import numpy as np
import pandas as pd
from patsy import dmatrices
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from statsmodels.tools.tools import add_constant
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings('ignore')

In [82]:
labels = pd.read_csv("../input/kdd-labels/labels.csv",sep = ":",header = None)
data   = pd.read_csv("../input/kdd-cup-10percent/kddcup.data_10_percent_corrected", names = labels.iloc[:,0].values)

In [83]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,output
0,0,tcp,http,SF,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [84]:
lb_make = LabelEncoder()
cat_columns = labels.loc[labels.iloc[:,1] == " symbolic.",0].values
data[cat_columns] = data[cat_columns].apply(lambda col: lb_make.fit_transform(col))

In [85]:
data.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,output
0,0,1,22,9,181,5450,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,9,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,1,22,9,239,486,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,19,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,1,22,9,235,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,8,8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,29,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,1,22,9,219,1337,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,39,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,1,22,9,217,2032,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,6,6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


In [86]:
def output_replace(df):
    replacements = {
        r'(smurf.|neptune.|back.|teardrop.|pod.|land.)' : 'dos',
        r'(normal.)' : 'normal',
        r'(satan.|ipsweep.|portsweep.|nmap.)' : 'probe',
        r'(warezclient.|guess_passwd.|warezmaster.|imap.|ftp_write.|multihop.|phf.|spy.)' : 'r2l',
        r'(buffer_overflow.|rootkit.|loadmodule.|perl.)' : 'u2r'
    }
    df.replace(replacements, regex=True, inplace=True)

In [87]:
def ffr(df, t):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    df_new = df.iloc[:,:-1]
    var_d = []
    for clas in classes:
        temp = df.loc[df['output'] == clas]
        #print("is unique ", temp.output.unique())
        temp = temp.iloc[:,:-1].values
        #print("temp = ", temp)
        min_max_scaler = preprocessing.MinMaxScaler()
        temp_scaled = min_max_scaler.fit_transform(temp)
        #print("temp_scaled = ", temp_scaled)
        mean_ind = temp_scaled.mean(axis = 0)
        #print("mean_ind = ", mean_ind)
        var_d_f = np.square(mean_ind - temp).mean(axis = 0)
        #print("var_d_f_f = ", var_d_f)
        var_d.append(var_d_f)
    var_d = np.array(var_d)
    #print(var_d)
    var_d_mean = var_d.mean(axis = 0)
    
    
#     var = np.zeros(len(mean_means))
#     for i in means:
#         var += np.square(i - mean_means)
#     var /= len(mean_means)
#     #print(var)
    indexes = list(np.argsort(var_d_mean))
    filtered_indexes = indexes[:t]
    filtered_indexes.append(41)
    filtered_data = df.iloc[:, filtered_indexes]
    return filtered_data

In [88]:
def data_create(df):
    classes = ['normal', 'u2r', 'dos', 'r2l', 'probe']
    print(np.unique(df['output'], return_counts=True))
    new_df = df.loc[df['output'] == 'normal'].sample(n = 52)
    for i in range(1, 5):
        new_df = new_df.append(df.loc[df['output'] == classes[i]].sample(n = 52), ignore_index = True)
    #print(new_df)
    new_df = new_df.sample(frac=1)
    return new_df

In [89]:
def naive_bayes(df, k):
    #print(df.head())
    X = df.iloc[:,:-1].values
    Y = df.iloc[:, -1].values
    kf = KFold(n_splits=k)
    itr = 0
    accuracy_total = []
    precision_total = []
    recall_total = []
    f1_total = []
    acc_overall_total = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index] 
        #print("Y_train = " ,np.unique(y_train, return_counts=True))
        #print("y_test = " ,np.unique(y_test, return_counts=True))
        mnb = MultinomialNB()
        mnb.fit(X_train, y_train)
        preds = mnb.predict(X_test)
        Accuracy = accuracy_score(y_test, preds)
        precision, recall, f1, support = precision_recall_fscore_support(y_test, preds)
        cm = confusion_matrix(y_test, preds)
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        accuracy_total.append(cm.diagonal())
        precision_total.append(precision)
        recall_total.append(recall)
        f1_total.append(f1)
        acc_overall_total.append(Accuracy)
#     print("acc total = ", [len(x) for x in accuracy_total])
#     print("prec total = ",[len(x) for x in precision_total])
#     print("recall total = ",[len(x) for x in recall_total])
#     print("f1 total = ",[len(x) for x in f1_total])
    
    print("accuracy = ", np.array(accuracy_total).mean(axis=0))
    print('Precision: ', np.array(precision_total).mean(axis=0))
    print('Recall: ', np.array(recall_total).mean(axis=0))
    print('F1: ', np.array(f1_total).mean(axis=0))
    print("Accuracy overall = ", np.array(acc_overall_total).mean())
    
    return np.array(accuracy_total).mean(axis=0), np.array(f1_total).mean(axis=0)

In [90]:
def knn_classify(df, k, folds):
    X = df.iloc[:,:-1].values
    Y = df.iloc[:, -1].values
    knn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
    knn.fit(X_train, y_train)
    preds = knn.predict(X_test)
    Accuracy = accuracy_score(y_test, preds)
    precision, recall, f1, support = precision_recall_fscore_support(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    print("accuracy = ", cm.diagonal())
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1: ', f1)
    print("Accuracy overall = ", Accuracy)
    
    return cm.diagonal(), f1

    
#     kf = KFold(n_splits=folds)
#     itr = 0
#     accuracy_total = []
#     precision_total = []
#     recall_total = []
#     f1_total = []
#     acc_overall_total = []
#     for train_index, test_index in kf.split(X):
#         print("new fold")
#         X_train, X_test = X[train_index], X[test_index]
#         y_train, y_test = Y[train_index], Y[test_index]
#         knn.fit(X_train, y_train)
#         print("fitted predictiing")
#         preds = knn.predict(X_test)
#         print("predicted")
#         Accuracy = accuracy_score(y_test, preds)
#         precision, recall, f1, support = precision_recall_fscore_support(y_test, preds)
#         cm = confusion_matrix(y_test, preds)
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         accuracy_total.append(cm.diagonal())
#         precision_total.append(precision)
#         recall_total.append(recall)
#         f1_total.append(f1)
#         acc_overall_total.append(Accuracy)
        
#     print("accuracy = ", np.array(accuracy_total).mean(axis=0))
#     print('Precision: ', np.array(precision_total).mean(axis=0))
#     print('Recall: ', np.array(recall_total).mean(axis=0))
#     print('F1: ', np.array(f1_total).mean(axis=0))
#     print("Accuracy overall = ", np.array(acc_overall_total).mean())

    


## Original data results

In [91]:
output_replace(data)
print("Naive Bayes")
original_ueq_nb_acc, original_ueq_nb_f1 = naive_bayes(data, 3)
print("KNN")
original_ueq_knn_acc, original_ueq_knn_f1 = knn_classify(data, 9, 5)

Naive Bayes
accuracy =  [0.97638564 0.59038061 0.50386663 0.34103704 0.63018491]
Precision:  [0.94527346 0.977603   0.37912621 0.33369866 0.33624315]
Recall:  [0.97638564 0.59038061 0.50386663 0.34103704 0.63018491]
F1:  [0.96017058 0.69493671 0.40553182 0.33403091 0.33908879]
Accuracy overall =  0.8676392579529094
KNN
accuracy =  [0.99983391 0.99840895 0.90797546 0.95515695 0.45454545]
Precision:  [0.9992977  0.99728289 0.98143236 0.94247788 1.        ]
Recall:  [0.99983391 0.99840895 0.90797546 0.95515695 0.45454545]
F1:  [0.99956573 0.9978456  0.94327597 0.94877506 0.625     ]
Accuracy overall =  0.9986336723850008


## reduced data for original distribution of classes

In [92]:
ffr_nb_acc_ueq = []
ffr_nb_f1_ueq = []
ffr_knn_acc_ueq = []
ffr_knn_f1_ueq = []
for i in range(10, 31, 10):
    new_dat = ffr(data, i)
    print("features = ", i)
    print("------------------------------Naive Bayes----------------------------")
    acc, f1 = naive_bayes(new_dat, 3)
    ffr_nb_acc_ueq.append(acc)
    ffr_nb_f1_ueq.append(f1)
    print("-------------------------------KNN-----------------------------------")
    acc, f1 = knn_classify(new_dat, 9, 5)
    ffr_knn_acc_ueq.append(acc)
    ffr_knn_f1_ueq.append(f1)

features =  10
------------------------------Naive Bayes----------------------------
accuracy =  [0.99996628 0.33779294 0.33333333 0.33333333 0.33333333]
Precision:  [0.79378198 0.734917   0.33333333 0.33333333 0.33333333]
Recall:  [0.99996628 0.33779294 0.33333333 0.33333333 0.33333333]
F1:  [0.87186063 0.34212587 0.33333333 0.33333333 0.33333333]
Accuracy overall =  0.7935756669045873
-------------------------------KNN-----------------------------------
accuracy =  [0.99989799 0.51023044 0.33248408 0.02173913 0.        ]
Precision:  [0.88649107 0.98552894 0.82857143 0.5        0.        ]
Recall:  [0.99989799 0.51023044 0.33248408 0.02173913 0.        ]
F1:  [0.9397856  0.67236331 0.47454545 0.04166667 0.        ]
Accuracy overall =  0.896310915439502
features =  20
------------------------------Naive Bayes----------------------------
accuracy =  [0.94894758 0.47932247 0.39128596 0.33333333 0.48575712]
Precision:  [0.81707762 0.67134868 0.66666667 0.33333333 0.5374677 ]
Recall:  [0.9

## Testing for data with equal distribution (original)

In [93]:
new_dat = data_create(data)
orig_nb_eq_distr_acc = naive_bayes(new_dat, 10)
orig_knn_eq_distr_acc = knn_classify(new_dat, 9, 5)

(array(['dos', 'normal', 'probe', 'r2l', 'u2r'], dtype=object), array([391458,  97278,   4107,   1126,     52]))
accuracy =  [0.84       0.52904762 0.91333333 0.0825     0.85166667]
Precision:  [0.51131258 0.62261905 0.76669913 0.4        0.77555556]
Recall:  [0.84       0.52904762 0.91333333 0.0825     0.85166667]
F1:  [0.62108059 0.56157287 0.82448662 0.13412698 0.77710264]
Accuracy overall =  0.6384615384615384
accuracy =  [0.93333333 0.6        0.85714286 0.75       0.28571429]
Precision:  [0.93333333 0.75       0.6        1.         0.22222222]
Recall:  [0.93333333 0.6        0.85714286 0.75       0.28571429]
F1:  [0.93333333 0.66666667 0.70588235 0.85714286 0.25      ]
Accuracy overall =  0.7115384615384616


### Reduced feature equal distribution

In [94]:
ffr_nb_acc_eq = []
ffr_nb_f1_eq = []
ffr_knn_acc_eq = []
ffr_knn_f1_eq = []
for i in range(10, 31, 10):
    new_dat_temp = ffr(new_dat, i)
    print("features = ", i)
    print("---------------------naive bayes---------------------")
    acc, f1 = naive_bayes(new_dat_temp, 10)
    ffr_nb_acc_eq.append(acc)
    ffr_nb_f1_eq.append(f1)
    print("---------------------KNN---------------------------")
    acc, f1 = knn_classify(new_dat_temp, 9, 5)
    ffr_knn_acc_eq.append(acc)
    ffr_knn_f1_eq.append(f1)

features =  10
---------------------naive bayes---------------------
accuracy =  [0.49 0.   0.3  0.3  0.  ]
Precision:  [0.40938462 0.         0.03704348 0.03149586 0.        ]
Recall:  [0.49 0.   0.3  0.3  0.  ]
F1:  [0.24263273 0.         0.06593407 0.05607692 0.        ]
Accuracy overall =  0.13846153846153847
---------------------KNN---------------------------
accuracy =  [1.         0.         0.33333333 0.1        0.        ]
Precision:  [0.22727273 0.         1.         0.33333333 0.        ]
Recall:  [1.         0.         0.33333333 0.1        0.        ]
F1:  [0.37037037 0.         0.5        0.15384615 0.        ]
Accuracy overall =  0.3076923076923077
features =  20
---------------------naive bayes---------------------
accuracy =  [0.49       0.29785714 0.59619048 0.80297619 0.47297619]
Precision:  [0.53080808 0.37666667 0.67313853 0.65291667 0.7       ]
Recall:  [0.49       0.29785714 0.59619048 0.80297619 0.47297619]
F1:  [0.36598291 0.31873016 0.56275336 0.60876875 0.518